In [13]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint
from statsmodels.api import OLS
import numpy as np
import xgboost as xgb
import statsmodels.api as sm

#### Load Data

In [22]:
# Download Historical Data
start_date = "2023-01-01"
end_date = "2024-01-01"

# Get the Gold Future and Silver Future
xauusd = yf.download("GC=F", start=start_date, end=end_date)
xagusd = yf.download("SI=F", start=start_date, end=end_date)

# Prepare the data
data = pd.DataFrame({
    'Gold': xauusd['Adj Close']['GC=F'],
    'Silver': xagusd['Adj Close']['SI=F']
}).dropna()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


#### Cointegrate & Correlation

needs long enough of time to see the relationship

In [5]:
# Create cointegration dataframe to see the correlation
coint_df = pd.concat([data['Gold'], data['Silver']], axis=1)

# Correlation Test
correlation = coint_df.corr()
print(f"Correlation between Gold and Silver: {correlation.loc['Gold', 'Silver']}")

# Cointegration Test
score, p_value, _ = coint(coint_df['Gold'], coint_df['Silver'])
print(f"Cointegration p-value: {p_value}")

del coint_df

Correlation between Gold and Silver: 0.8319010117243449
Cointegration p-value: 0.668497624232878


In [6]:
def split_train_test_by_year(data, date_column, target_column, test_years):
    """
    Splits the dataset into train and test sets based on years.
    
    Parameters:
        data (pd.DataFrame): The dataset containing features and target.
        date_column (str): The name of the column containing dates.
        target_column (str): The name of the target column.
        test_years (int): The number of years to include in the test set.
        
    Returns:
        X_train, X_test, Y_train, Y_test
    """
    # Ensure the date column is in datetime format
    data[date_column] = pd.to_datetime(data[date_column])
    
    # Extract the year from the date column
    data['Year'] = data[date_column].dt.year
    
    # Determine the threshold year for the split
    max_year = data['Year'].max()
    threshold_year = max_year - test_years + 1  # First year to include in the test set
    
    # Split data into train and test sets
    train_data = data[data['Year'] < threshold_year]
    test_data = data[data['Year'] >= threshold_year]
    
    # Separate features (X) and target (Y)
    X_train = train_data.drop(columns=[target_column, 'Year'])
    Y_train = train_data[target_column]
    X_test = test_data.drop(columns=[target_column, 'Year'])
    Y_test = test_data[target_column]
    
    return X_train, X_test, Y_train, Y_test

In [7]:
tmp_df = data.copy()
tmp_df.reset_index(inplace=True)
X_train, X_test, Y_train, Y_test = split_train_test_by_year(tmp_df, "Date", "Silver", 1)

In [24]:
# OLS is Simple Linear Regression
model = OLS(data['Gold'], data['Silver']).fit()

# Get the hedge ratio to make both assests be in the same scale
hedge_ratio = np.array(model.params)

# Calculate Spread (difference between two assets)
data['Spread'] = data['Gold'] - (hedge_ratio * data['Silver'])

In [25]:
def calculate_half_life(spread):
    df_spread = pd.DataFrame(spread, columns=['Spread'])
    spread_lag = df_spread['Spread'].shift(1)
    spread_lag.iloc[0] = spread_lag.iloc[1]
    spread_return = df_spread['Spread'] - spread_lag
    spread_return.iloc[0] = spread_return.iloc[1]
    spread_lag2 = sm.add_constant(spread_lag)
    model = sm.OLS(spread_return, spread_lag2)
    res = model.fit()
    half_life = round(-np.log(2) / res.params[1], 0)
    return half_life

calculate_half_life(data['Spread'])

/var/folders/k_/3s9bjxk17ss2bty8k9tjwjrc0000gn/T/ipykernel_22571/4138012894.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  half_life = round(-np.log(2) / res.params[1], 0)


np.float64(11.0)

In [20]:
def calculate_zscore(spread, window=21):
    spread_series = pd.Series(spread)
    mean = spread_series.rolling(center=False, window=window).mean()
    std = spread_series.rolling(center=False, window=window).std()
    x = spread_series.rolling(center=False, window=window).mean()
    zscore = (x - mean) / std
    return zscore

zscore = calculate_zscore(data["Spread"])

In [21]:
zscore.describe()

count    1238.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: Spread, dtype: float64

In [9]:
# Calculate mean and standard deviation for calculate Z-Score
mean_spread = data['Spread'].mean()
std_spread = data['Spread'].std()

# Calculate Z-Score
data['Z-Score'] = (data['Spread'] - mean_spread) / std_spread